In [384]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import re
from io import StringIO

# 1.0- Data Scraping

First we must pull the data from baskeball-reference.com

In [470]:
lebron = 'https://www.basketball-reference.com/players/j/jamesle01.html'
rose = 'https://www.basketball-reference.com/players/r/rosede01.html'
mj = 'https://www.basketball-reference.com/players/j/jordami01.html'
z = 'https://www.basketball-reference.com/players/z/zipsepa01.html'
players = [lebron,rose,mj,z]

In [471]:
chromedriver = "/home/michael/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [472]:
driver = webdriver.Chrome(chromedriver)
driver.get(players[1])
time.sleep(3)
expand = driver.find_element_by_xpath('//button[@id="meta_more_button"]')
time.sleep(1)
expand.click()

## 1.1 Get the Player Info

In [473]:
name = driver.find_element_by_xpath('//h1[@itemprop="name"]').text
name

'Derrick Rose'

In [474]:
height = driver.find_element_by_xpath('//span[@itemprop="height"]').text
height

'6-3'

In [475]:
weight = driver.find_element_by_xpath('//span[@itemprop="weight"]').text
weight

'190lb'

In [476]:
info = driver.find_element_by_xpath('//div[@id="info"]').text
info = info.split('\n')
info #10 Recruit 11 Draft 

['Derrick Rose',
 'Derrick Martell Rose ▪ Twitter: drose',
 '(D-Rose)',
 'Position: Point Guard ▪ Shoots: Right',
 '6-3, 190lb (190cm, 86kg)',
 'Team: Cleveland Cavaliers',
 'Born: October 4, 1988 (Age: 29-113d) in Chicago, Illinois us',
 'College: University of Memphis',
 'High School: Simeon Career Academy in Chicago, Illinois',
 'Recruiting Rank: 2007 (5)',
 'Draft: Chicago Bulls, 1st round (1st pick, 1st overall), 2008 NBA Draft',
 'NBA Debut: October 28, 2008',
 'Experience: 8 years',
 '3x All Star',
 '2010-11 All-NBA',
 '2008-09 All-Rookie',
 '2010-11 MVP',
 '2008-09 ROY',
 '1',
 '25',
 '1',
 'Support us without the ads? Go Ad-Free.',
 'SUMMARY',
 '2017-18',
 'Career',
 'G',
 '10',
 '480',
 'PTS',
 '12.7',
 '19.3',
 'TRB',
 '2.3',
 '3.7',
 'AST',
 '1.7',
 '5.9',
 'FG%',
 '48.5',
 '45.1',
 'FG3%',
 '20.0',
 '29.7',
 'FT%',
 '90.3',
 '82.2',
 'eFG%',
 '50.0',
 '47.4']

In [485]:
draft_regex = re.compile('Draft:')
draftstr = [x  for x in info if draft_regex.search(x)]
draftstr = draftstr[0].split(',')
draftstr

['Draft: Chicago Bulls',
 ' 1st round (1st pick',
 ' 1st overall)',
 ' 2008 NBA Draft']

In [487]:
draftnbr = draftstr[2].strip().split(' ')
draftnbr = draftnbr[0]
draftnbr = int(draftnbr[:-2])
draftnbr

1

In [492]:
draftyr = draftstr[3].strip().split(' ')
draftyr = int(draftyr[0])
draftyr

2008

In [497]:
recruit_regex = re.compile('Recruiting')
recruitstr = [x  for x in info if recruit_regex.search(x)]
recruitstr = recruitstr[0].split(' ')
recruitnbr = recruitstr[-1]
recruitnbr = int(recruitnbr[1:-1])
recruitnbr

5

## 1.2 Get the VORP Yearly Log

In [410]:
advstatsxpath = driver.find_element_by_xpath('//table[@id="advanced"]').text
advstatsxpath = advstatsxpath.replace('  ','')
advstatsxpath

'Season Age Tm Lg Pos G MP PER TS% 3PAr FTr ORB% DRB% TRB% AST% STL% BLK% TOV% USG% OWS DWS WS WS/48 OBPM DBPM BPM VORP\n2008-09 20 CHI NBA PG 81 3000 16.0 .516 .060 .207 3.7 8.4 6.1 28.8 1.1 0.4 13.3 22.6 3.4 1.5 4.9 .078 0.9 -1.5 -0.6 1.1\n2009-10 21 CHI NBA PG 78 2871 18.6 .532 .044 .246 2.6 8.7 5.7 30.3 1.0 0.7 12.5 27.2 3.5 2.5 6.0 .100 1.7 -1.3 0.5 1.8\n2010-11 22 CHI NBA PG 81 3026 23.5 .550 .241 .348 3.2 9.4 6.4 38.7 1.5 1.3 13.1 32.2 8.3 4.8 13.1 .208 5.8 0.1 5.9 6.0\n2011-12 23 CHI NBA PG 39 1375 23.0 .532 .249 .344 2.1 8.4 5.3 40.3 1.4 1.4 12.9 30.5 4.1 2.0 6.0 .211 5.5 -0.6 4.9 2.4\n2013-14 25 CHI NBA PG 10 311 9.7 .446 .287 .195 3.3 8.3 5.9 26.2 0.9 0.2 16.0 31.5 -0.6 0.4 -0.2 -0.036 -2.8 -2.3 -5.1 -0.2\n2014-15 26 CHI NBA PG 51 1530 15.9 .493 .325 .224 2.6 8.7 5.7 30.7 1.2 0.8 14.9 31.7 0.0 1.2 1.2 .038 1.1 -2.3 -1.3 0.3\n2015-16 27 CHI NBA PG 66 2097 13.4 .479 .143 .171 2.4 8.8 5.6 25.5 1.1 0.5 13.4 27.3 -0.8 1.2 0.4 .009 -1.2 -2.1 -3.3 -0.7\n2016-17 28 NYK NBA PG 64 208

In [421]:
dfadvstats = pd.read_csv(StringIO(advstatsxpath),sep=' ')
dfadvstats = dfadvstats.dropna()
dfadvstats['player'] = name
dfadvstats

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,player
0,2008-09,20,CHI,NBA,PG,81.0,3000.0,16.0,0.516,0.060,...,22.6,3.4,1.5,4.9,0.078,0.9,-1.5,-0.6,1.1,Derrick Rose
1,2009-10,21,CHI,NBA,PG,78.0,2871.0,18.6,0.532,0.044,...,27.2,3.5,2.5,6.0,0.100,1.7,-1.3,0.5,1.8,Derrick Rose
2,2010-11,22,CHI,NBA,PG,81.0,3026.0,23.5,0.550,0.241,...,32.2,8.3,4.8,13.1,0.208,5.8,0.1,5.9,6.0,Derrick Rose
3,2011-12,23,CHI,NBA,PG,39.0,1375.0,23.0,0.532,0.249,...,30.5,4.1,2.0,6.0,0.211,5.5,-0.6,4.9,2.4,Derrick Rose
4,2013-14,25,CHI,NBA,PG,10.0,311.0,9.7,0.446,0.287,...,31.5,-0.6,0.4,-0.2,-0.036,-2.8,-2.3,-5.1,-0.2,Derrick Rose
5,2014-15,26,CHI,NBA,PG,51.0,1530.0,15.9,0.493,0.325,...,31.7,0.0,1.2,1.2,0.038,1.1,-2.3,-1.3,0.3,Derrick Rose
6,2015-16,27,CHI,NBA,PG,66.0,2097.0,13.4,0.479,0.143,...,27.3,-0.8,1.2,0.4,0.009,-1.2,-2.1,-3.3,-0.7,Derrick Rose
7,2016-17,28,NYK,NBA,PG,64.0,2082.0,17.0,0.530,0.061,...,25.7,2.4,0.6,3.0,0.068,0.2,-2.2,-1.9,0.0,Derrick Rose
8,2017-18,29,CLE,NBA,PG,10.0,235.0,12.5,0.564,0.152,...,26.2,0.0,0.0,-0.1,-0.012,-3.7,-3.9,-7.6,-0.3,Derrick Rose


## 1.3 College Stats